In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
import torch
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [2]:
file_path = "attention.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

print(len(docs))

15


In [3]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and
{'source': 'attention.pdf', 'page': 0}


In [4]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/home/mudaliar.k/github/Structured-Abstracts/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=hf
)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [6]:
llm = HuggingFacePipeline.from_model_id(
    model_id="nvidia/Nemotron-Mini-4B-Instruct",
    task="text-generation",
    device=0, 
    pipeline_kwargs={
        "top_k": 50,
        "temperature": 0.1
    },
)
llm.invoke("Hugging Face is")
#print(chat.invoke("What is the capital of France?"))

tokenizer_config.json:   0%|          | 0.00/180k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/18.1M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/8.38G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/8.38G [00:00<?, ?B/s]

/home/mudaliar.k/github/Structured-Abstracts/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/mudaliar.k/github/Structured-Abstracts/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mudaliar.k/github/Structured-Abstracts/.venv/lib/python3.11/site-packages/transformers/models/nemotron/modeling_nemotron.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


'Hugging Face is a community-driven platform for AI and ML. It provides a wide range of'

In [7]:
prompt = 'What is the paper about?'

In [8]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "what is the topic of this paper?"})

results

/home/mudaliar.k/github/Structured-Abstracts/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/mudaliar.k/github/Structured-Abstracts/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
len(results['answer'])

2768